# Experiment Initialization

After capturing an experiment video (or series of frames), in order to run the system simulation on it, one must first initialize this experiment.
The experiment initialization process is the process in which the YOLO model predicts the worm head position in every frame of the experiment video.
These worm head positions are afterwards logged into a file. 

This initialization step is designed to address two important issues:
1.  Running a simulation on a sequence of frames (images) that are read from the disk is slow, since the process loading large amount of images from the disk is relatively slow.
2.  Given an image, running the YOLO model on that image and detecting worm's head position is also relatively slow.

Therefore, instead of performing the above two steps every time we want to run a simulation on some experiment, we instead perform these steps only *ONCE*. On each frame, we run the YOLO predictor and detect the worm's head position. Afterwards, we log the detected head coordinates into a log file (bboxes.csv). As result, consequent simulations of this experiment can simply read the log file to obtain worm head positions, instead of running YOLO again. Also, loading the images fro the disk is no longer required, since all the relevant information is in the log.


In [ ]:
# fix imports
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from wtracker.sim import *
from wtracker.sim.sim_controllers import *
from wtracker.utils.path_utils import Files, join_paths
from wtracker.utils.gui_utils import UserPrompt
from wtracker.utils.frame_reader import FrameReader
import cv2 as cv

### Input and output directories

In [ ]:
################################ User Input ################################

# define the path to the folder containing the frame images of the experiment
# if none, the user will be prompted to select the folder
images_folder = None

# define the path into which the analysis of the experiment will be saved,
# including bounding box detections at each frame (bboxes.csv)
# if none, the user will be prompted to select the folder
output_folder = None

############################################################################

if images_folder is None:
    images_folder = UserPrompt.open_directory("Select the folder containing the frame images")

if output_folder is None:
    output_folder = UserPrompt.open_directory("Select the folder to save the output analysis to")

print(f"input images folder: {images_folder}")
print(f"output folder: {output_folder}")

In [ ]:
from typing import Callable, Union

################################ User Input ################################

# select file extension of the frame images
frame_file_extension = "bmp"

# define the function by which the image files are ordered. 
# The function should take a string as input and return a string or an integer.
filename_sorting_func: Callable[[str], Union[str, int]] = lambda name: int(name.split("-")[-1].split(".")[0])

############################################################################

# read the frame images from the provided directory
files = Files(
    extension=frame_file_extension,
    directory=images_folder,
    sorting_key=filename_sorting_func,
    scan_dirs=False,
    return_full_path=False,
)

frame_files = list(files)
reader = FrameReader(images_folder, frame_files, read_format=cv.IMREAD_GRAYSCALE)

### Experiment, timing settings and YOLO configuration

In [ ]:
################################ User Input ################################

# create the experiment config
# this configuration differs between different experiments
experiment_config = ExperimentConfig.from_frame_reader(
    reader,
    name="experiment3",
    frames_per_sec=60,
    px_per_mm=88,
    init_position=(132, 1032),
)

# initialize the timing configuration
# changing this config changes the simulation settings, should remain the same
# between different experiments
time_config = TimingConfig(
    imaging_time_ms=200,
    pred_time_ms=40,
    moving_time_ms=50,
    camera_size_mm=(4, 4),
    micro_size_mm=(0.32, 0.32),
    experiment_config=experiment_config,
)

############################################################################

# initialize logging settings
log_config = LogConfig(
    root_folder=output_folder,
    save_mic_view=False,  # whether to save images of the microscope view
    save_cam_view=False,  # whether to save images of the camera view
    save_wrm_view=True,  # whether to save images of the detected worms. These images are later used to calculate precise error during analysis. can be disabled to save disk space.
    save_err_view=True,  # whether to save camera view of frames when no object is detected. can be disabled to save disk space.
    bbox_file_name="init_bboxes.csv", 
)

In [ ]:
################################ User Input ################################

# specify inference device for the YOLO model. If GPU is available, it is recommended to select `cuda` option
device = "cpu"  # cpu or cuda

# specify the path to the YOLO model weights.
# If no path is specified, the user will be prompted to select a file.
yolo_model_path = None

############################################################################

if yolo_model_path is None:
    yolo_model_path = UserPrompt.open_file("Select YOLO model", [("pytorch file", "*.pt"), ("ONNX model", "*.onnx")])

# create the YOLO configuration
yolo_config = YoloConfig(
    model_path=yolo_model_path,
    device=device,
    verbose=True,  # controls the verbosity of the YOLO predictor. No need to be set to True since there is a progress bar anyways.
    pred_kwargs={
        "imgsz": 384,  # imgsz should be the same size on which the model was trained.
        "conf": 0.1,  # minimum confidence threshold for detection of worm head.
    },
)

In [ ]:
# save config files
experiment_config.save_json(join_paths(output_folder, "exp_config.json"))
log_config.save_json(join_paths(output_folder, "log_config.json"))
time_config.save_json(join_paths(output_folder, "time_config.json"))
yolo_config.save_json(join_paths(output_folder, "yolo_config.json"))

### Run YoloController and save log file

The YoloController expects the simulator to receive experiment images as input.   
This controller runs the YOLO predictor on the input images and predicts the bounding box around worm's head in each such image.

Note, that executing the cells below might take very long time, up to few hours.

In [ ]:
# create YoloController of the simulation that runs in real time and detects worms heap positions from the input frames
yolo_controller = YoloController(time_config, yolo_config)

# wrap the YoloController with a LoggingController to log the simulation data
# this controller logs the detected head positions into bboxes.csv file, which could be used afterwards
# by different controllers, without the need to run the YOLO model again
controller = LoggingController(yolo_controller, log_config)

In [ ]:
# create motor controller which controls the motion of the platform 
motor_controller = SineMotorController(time_config)

# create the simulator
sim = Simulator(
    time_config,
    experiment_config,
    controller,
    reader=reader,
    motor_controller=motor_controller,
)

In [ ]:
# run simulation and save log results
sim.run(visualize=False, wait_key=False)

### Extract the background of the experiment

In [ ]:
from wtracker.dataset.bg_extractor import BGExtractor

extractor = BGExtractor(reader)

################################ User Input ################################

# calculate the background of the experiment
background = extractor.calc_background(
    num_probes=1000,  # num frames from from which the background will be calculated
    sampling="uniform",  # sampling method for the frames. Options: "uniform", "median"
    method="median",  # method to calculate the background. Options: "mean", "median"
)

############################################################################

In [ ]:
import numpy as np

# save the background image
background_path = join_paths(output_folder, "background")
np.save(file=background_path, arr=background)